## <center>  Business insights using SQL database </center>
### In this notebook, I will be connecting to an SQL database and merging tables using various methods to develop relevant insights from the available data

In [1]:
import sqlite3

In [2]:
import pandas as pd

In [3]:
conn=sqlite3.connect("chinook.db")

In [4]:
cur=conn.cursor()

In [5]:
cur.execute("SELECT * FROM track LIMIT 5;")

In [6]:
cur.fetchall()

[(1,
  'For Those About To Rock (We Salute You)',
  1,
  1,
  1,
  'Angus Young, Malcolm Young, Brian Johnson',
  343719,
  11170334,
  0.99),
 (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, 0.99),
 (3,
  'Fast As a Shark',
  3,
  2,
  1,
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman',
  230619,
  3990994,
  0.99),
 (4,
  'Restless and Wild',
  3,
  2,
  1,
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman',
  252051,
  4331779,
  0.99),
 (5,
  'Princess of the Dawn',
  3,
  2,
  1,
  'Deaffy & R.A. Smith-Diesel',
  375418,
  6290521,
  0.99)]

####  Using cursor to perform sql queries does not produce easily readable results. Hence we will be using pandas to read sql queries from the connection.

In [7]:
query_table = """SELECT name, type 
           FROM sqlite_master 
           WHERE type IN ('table','view');"""

In [8]:
df_database=pd.read_sql_query(query_table,conn)

In [9]:
df_database

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


#### [This](https://github.com/akhilsali/SQL-for-business-problems/blob/master/chinook_schema.png) is the schema of the database which we will be using to solve the following problems. Take some time to understand the connections and details of all the tables.

### Problem 1 - Which genre sells most tracks?

In [10]:
query1="""WITH countryfilter AS (SELECT * FROM invoice_line
INNER JOIN invoice i on invoice_line.invoice_id = i.invoice_id
INNER JOIN customer c on c.customer_id=i.customer_id
WHERE c.country='USA')
SELECT g.name Genre,SUM(cf.quantity) Sold_quantity,(CAST(COUNT(*) AS FLOAT)/(SELECT COUNT(*) FROM countryfilter))*100 Percentage_sales 
FROM countryfilter cf
INNER JOIN track t ON t.track_id=cf.track_id
INNER JOIN genre g ON g.genre_id=t.genre_id
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 10;"""

In [11]:
pd.read_sql_query(query1,conn).set_index("Genre")

,Sold_quantity,Percentage_sales
Genre,,
Rock,561,53.377735
Alternative & Punk,130,12.369172
Metal,124,11.798287
R&B/Soul,53,5.042816
Blues,36,3.425309
Alternative,35,3.330162
Pop,22,2.093245
Latin,22,2.093245
Hip Hop/Rap,20,1.902950


#### From the table above, it is clear that Rock music had the highest sales, accounting for more than half of the total sale of  music tracks. 

### Problem 2 - Which employee generated most sales?

In [12]:
query2="""SELECT e.employee_id,e.first_name||" "||e.last_name Name,ROUND(SUM(i.total),2) generated_sales,ROUND(SUM(i.total)/(SELECT SUM(total) FROM invoice),2)*100 percentage_sales_generated,e.hire_date,e.country FROM invoice i
LEFT JOIN customer c ON c.customer_id=i.customer_id
LEFT JOIN employee e ON e.employee_id=c.support_rep_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 10;"""

Note that we will be using left join as out aim is to find out sales generated by each employee only and not the total sales

In [13]:
pd.read_sql_query(query2,conn).set_index('employee_id')

,Name,generated_sales,percentage_sales_generated,hire_date,country
employee_id,,,,,
3,Jane Peacock,1731.51,37.0,2017-04-01 00:00:00,Canada
4,Margaret Park,1584.00,34.0,2017-05-03 00:00:00,Canada
5,Steve Johnson,1393.92,30.0,2017-10-17 00:00:00,Canada


#### Although Jane generated most sales, we must also consider that she was hired earlier than the other employees. Lets verify their designations

In [14]:
pd.read_sql_query("""SELECT employee_id,title FROM employee;""",conn).set_index("employee_id")


,title
employee_id,
1,General Manager
2,Sales Manager
3,Sales Support Agent
4,Sales Support Agent
5,Sales Support Agent
6,IT Manager
7,IT Staff
8,IT Staff


### Problem 3 - Which country should the company focus on for increasing total revenue?

#### Some of the parameters we can use to select a country are:
* Number of customers
* Total Sales value
* Avg Sales per Customer
* Average Order Value

#### PARAMETER 1 - NUMBER OF CUSTOMERS PER COUNTRY
#### (Note that we will be creating a view to store each parameter query in the database so that we can merge them later as our final result. We have also added a sorter column to list all the countries with just one order as a single group 'other' at the bottom of our final result)

In [15]:
query_cust="""WITH custs AS (SELECT country,COUNT(DISTINCT(customer_id)) custnum
FROM customer
GROUP BY 1)
SELECT country,custnum,CASE
                        WHEN custnum=1 THEN 'other'
                        ELSE country
                        END AS country_cat,
                        CASE 
                        WHEN custnum=1 THEN 1 
                        ELSE 0 END AS sorter
from custs
ORDER BY 2 DESC;"""

In [16]:
pd.read_sql_query(query_cust,conn)

,country,custnum,country_cat,sorter
0,USA,13,USA,0
1,Canada,8,Canada,0
2,Brazil,5,Brazil,0
3,France,5,France,0
4,Germany,4,Germany,0
5,United Kingdom,3,United Kingdom,0
6,Czech Republic,2,Czech Republic,0
7,India,2,India,0
8,Portugal,2,Portugal,0
9,Argentina,1,other,1


#### Now lets add this table as a view into the database so that we can merge the parameters into a single table later. Note that we must always be cautious when making modifications to the database


In [17]:
cur.execute("""CREATE VIEW query_cust AS 
"""+query_cust)

#### PARAMETER 2 - TOTAL SALES BY COUNTRY

In [18]:
query_country="""SELECT c.country,SUM(i.total) total_sales FROM invoice i
INNER JOIN customer c ON c.customer_id=i.customer_id
GROUP BY c.country
ORDER BY 2 DESC;"""

In [19]:
pd.read_sql_query(query_country,conn)

,country,total_sales
0,USA,1040.49
1,Canada,535.59
2,Brazil,427.68
3,France,389.07
4,Germany,334.62
5,Czech Republic,273.24
6,United Kingdom,245.52
7,Portugal,185.13
8,India,183.15
9,Ireland,114.84


In [20]:
cur.execute("""CREATE VIEW query_country AS 
"""+query_country)

#### PARAMETER 3 - AVG CUSTOMER VALUE

In [21]:
query_custval="""WITH per_customer AS (SELECT country,SUM(i.total) value_per_cust
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY c.customer_id)
SELECT country,ROUND(AVG(value_per_cust),2) avg_cust_value
FROM per_customer
GROUP BY 1
ORDER BY 2 DESC;"""

In [22]:
pd.read_sql_query(query_custval,conn)

,country,avg_cust_value
0,Czech Republic,136.62
1,Ireland,114.84
2,Spain,98.01
3,Chile,97.02
4,Portugal,92.57
5,India,91.58
6,Brazil,85.54
7,Germany,83.66
8,United Kingdom,81.84
9,Australia,81.18


In [23]:
cur.execute("""CREATE VIEW query_custval AS 
"""+query_custval)

#### PARAMETER 4 - AVG ORDER VALUE

In [24]:
query_order="""SELECT country,AVG(i.total) avg_order
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY 1
ORDER BY 2 DESC;"""

In [25]:
pd.read_sql_query(query_order,conn)

,country,avg_order
0,Czech Republic,9.108000
1,Spain,8.910000
2,Ireland,8.833846
3,United Kingdom,8.768571
4,India,8.721429
5,Belgium,8.627143
6,Germany,8.161463
7,Australia,8.118000
8,Norway,8.030000
9,USA,7.942672


In [26]:
cur.execute("""CREATE VIEW query_order AS 
"""+query_order)

#### Now lets merge these views together to undertand the complete picture

In [27]:
final_query="""SELECT qcu.country_cat Country,SUM(qcu.custnum) Num_custs,ROUND(SUM(qco.total_sales),2) Total_Sales,ROUND(AVG(qcv.avg_cust_value),2) Avg_cust_value,ROUND(AVG(qo.avg_order),2) Avg_order_value  
FROM query_cust qcu
LEFT JOIN query_country qco ON qco.country=qcu.country
LEFT JOIN query_custval qcv ON qcv.country=qcu.country
LEFT JOIN query_order qo ON qo.country=qcu.country
GROUP BY 1
ORDER BY qcu.sorter,4 DESC,5 DESC;"""

In [28]:
pd.read_sql_query(final_query,conn).set_index('Country')

,Num_custs,Total_Sales,Avg_cust_value,Avg_order_value
Country,,,,
Czech Republic,2,273.24,136.62,9.11
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Brazil,5,427.68,85.54,7.01
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
USA,13,1040.49,80.04,7.94
France,5,389.07,77.81,7.78
Canada,8,535.59,66.95,7.05


#### Based on average amount spent by a customer and average order value, we can say that the company must focus on Czech Republic for increasing revenue.